# Building Your Own AI Research Agent


### About Me
- will brown from ai twitter (@willccbb)
- research lead @ prime intellect
- spent 2yrs @ morgan stanley doing LLM applications
- did phd @ columbia on multi-agent learning theory
- i work on agentic RL stuff (see [willccbb/verifiers](https://github.com/willccbb/verifiers) on github)


### About the Full Course (Production-Ready Agent Engineering: From MCP to RL)
- runs june 16 - july 4
- co-teaching with kyle corbitt (@corbtt), ceo of openpipe
- agent stuff AND rl stuff
    - two sides of the same coin
    - course starts with practical 
    - builds towards RL finetuning for OSS models
    - most patterns have analogues which can be applied to closed/API models

### Today's Lightning Lesson
- Basic definitions
- Function-calling and writing your own tools
- MCP crash course
- Connecting MCP servers to clients
- Scaffolding a "deep research"

### What's an "agent"?

No universally agreed-upon definition, but:
- A system that can take actions in an environment towards a goal
- Ability to dynamically adapt logic on the fly in response to observations
- Core contrast: agents vs. workflows/pipelines

Agents:
- Deep Research
- Claude Code
- Manus 

Non-agents:
- pre-fetch RAG
- fixed decision trees of LLMs


Reductive definition:
- "LLM with tool calls in a while loop"
- Must-read: [Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) from Anthropic


![AGENTS](images/agent.webp)

### What's an "environment"?

Most basic example:
- system prompt
- set of tools

Simple version with no fancy libraries:
- OpenAI client (can use any model/provider, or locally-hosted endpoint)
- search tool + fetch tool

In [35]:
import os
from openai import OpenAI

model_name = "gpt-4.1"
base_url = "https://api.openai.com/v1"
client = OpenAI(base_url=base_url, api_key=os.getenv("OPENAI_API_KEY"))


system_prompt = """
You are a helpful assistant that can answer questions and help with tasks, such as drafting short research reports. Cite your sources when relevant. 

You have access to the following tools:

- search(query: str) -> str: Searches the web and returns summaries of top results.
- fetch(url: str) -> str: Fetches the content of a given URL and returns it as a markdown page.

You may call one tool per turn, for up to 10 turns, before giving your final answer.
In each turn, you should respond in the following format:

<think>
[your thoughts here]
</think>
<tool>
JSON with the following fields:
- name: The name of the tool to call
- args: A dictionary of arguments to pass to the tool (must be valid JSON)
</tool>

When you are done, give your final answer in the following format:

<answer>
[your final answer here]
</answer>
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the latest news in the NBA?"},
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages, # type: ignore
)

response = response.choices[0].message.content # type: ignore
print(response)


<think>
I will search for the latest NBA news headlines to provide a current summary.
</think>
<tool>
{"name": "search", "args": {"query": "latest NBA news"}}
</tool>


In [36]:
# Basic search tool

def search(query: str) -> str:
    """Searches the web and returns summaries of top results.
    
    Args:
        query: The search query string

    Returns:
        Formatted string with bullet points of top 10 results, each with title, source, url, and brief summary

    Examples:
        {"query": "who invented the lightbulb"} -> ["Thomas Edison (1847-1931) - Inventor of the lightbulb", ...]
        {"query": "what is the capital of France"} -> ["Paris is the capital of France", ...]
        {"query": "when was the Declaration of Independence signed"} -> ["The Declaration of Independence was signed on July 4, 1776", ...]
    """

    try:
        from brave import Brave
        # set BRAVE_API_KEY in your environment
        brave = Brave()
        results = brave.search(q=query, count=10, raw=True) # type: ignore
        web_results = results.get('web', {}).get('results', []) # type: ignore
        
        if not web_results:
            return "No results found"

        summaries = []
        for r in web_results:
            if 'profile' not in r:
                continue
            header = f"{r['profile']['name']} ({r['profile']['long_name']})"
            title = r['title']
            snippet = r['description']
            url = r['url'] 
            summaries.append(f"•  {header}\n   {title}\n   {snippet}\n   {url}")

        return "\n\n".join(summaries)
    except Exception as e:
        return f"Error: {str(e)}"
# test
results = search("latest basketball scores")
print(results)

•  Flashscoreusa (flashscoreusa.com)
   Basketball Livescore, Basketball Results | Flashscore - NBA, Euroleague, NCAA
   <strong>Basketball</strong> livescore on Flashscore offers all the <strong>latest</strong> <strong>basketball</strong> results from more than 500+ <strong>basketball</strong> leagues all around the world including NBA, Euroleague, NCAA and more. Find all today&#x27;s/tonight&#x27;s <strong>basketball</strong> <strong>scores</strong> on Flashscore.
   https://www.flashscoreusa.com/basketball/

•  ESPN (Entertainment and Sports Programming Network)
   NBA Scores, 2024-25 Season - ESPN
   Live <strong>scores</strong> for every 2024-25 NBA season game on ESPN. Includes box <strong>scores</strong>, video highlights, play breakdowns and updated odds.
   https://www.espn.com/nba/scoreboard

•  Ncaa (ncaa.com)
   NCAA college basketball scores | NCAA.com
   Live college <strong>basketball</strong> <strong>scores</strong>, schedules and rankings from NCAA Division I men&#x27;

In [37]:
# fetch tool for URL contents

def fetch(url: str) -> str:
    """Fetches the content of a given URL and returns it as a markdown page.
    
    Args:
        url: The URL to fetch the content from

    Returns:
        A markdown page with the content of the URL.
    """
    import requests
    from markdownify import markdownify
    
    response = requests.get(url)
    response.raise_for_status()
    return markdownify(response.text)
content = fetch("https://www.ncaa.com/scoreboard/basketball-men/d1")
print(content)

NCAA college basketball scores | NCAA.com


[Skip to main content](#main-content)

[View All Scores](#)

### TRENDING:

### Follow live

[Women's College World Series begins](https://www.ncaa.com/live-updates/softball/d1/live-updates-2025-womens-college-world-series)

[DIII softball finals](https://www.ncaa.com/liveschedule)

[🏃‍♀️ DI track & field first rounds](https://www.ncaa.com/live-updates/trackfield-outdoor-women/d1/live-updates-2025-di-track-and-field-championships)

[🤔 Toughest DI baseball regionals](https://www.ncaa.com/news/baseball/article/2025-05-28/toughest-regional-and-national-champion-picks-2025-ncaa-baseball-tournament)

[NCAA.com](/)

* [Live Video](/liveschedule)
* Sports

  ### Sports

  #### Fall

  + [Cross Country - Men](/sports/cross-country-men)
  + [Cross Country - Women](/sports/cross-country-women)
  + Cross Country

    [MMen](/sports/cross-country-men)[WWomen](/sports/cross-country-women)
  + [Field Hockey](/sports/fieldhockey)
  + [Football](/sports/foot

In [38]:
# basic tool parsing -- universal but less robust, use instructor/outlines/API-provider parsing for more anything production-grade
import re
import json

def parse_thinking_from_response(response: str) -> str | None:
    """Parse a thinking from a response."""
    thinking = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if thinking:
        return thinking.group(1)
    return None

def parse_tool_from_response(response: str) -> dict | None:
    """Parse a tool from a response."""
    tool_call = re.search(r'<tool>(.*?)</tool>', response, re.DOTALL)
    if tool_call:
        return json.loads(tool_call.group(1))
    return None

def parse_answer_from_response(response: str) -> str | None:
    """Parse an answer from a response."""
    answer = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if answer:
        return answer.group(1)
    return None

def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""
    if tool_call['name'] == 'search':
        return search(tool_call['args']['query'])
    elif tool_call['name'] == 'fetch':
        return fetch(tool_call['args']['url'])
    else:
        return f"Error: Tool {tool_call['name']} not found"
    
# test
tool_call = {"name": "search", "args": {"query": "latest basketball scores"}}


example_response = """
<think>
I'll do a web search to retrieve the most recent news headlines and updates related to the NBA.
</think>
<tool>
{"name": "search", "args": {"query": "latest NBA news"}}
</tool>
"""

tool_call = parse_tool_from_response(example_response) # type: ignore
print(tool_call)

{'name': 'search', 'args': {'query': 'latest NBA news'}}


In [39]:
tool_result = call_tool(tool_call) # type: ignore
print(tool_result)

•  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong> <strong>News</strong>: Your source of the most updated official <strong>NBA</strong> <strong>news</strong>. Stay current on the league, team and player <strong>news</strong>, scores, stats, standings from <strong>NBA</strong>.
   https://www.nba.com/news

•  ESPN (Entertainment and Sports Programming Network)
   NBA on ESPN - Scores, Stats and Highlights
   Visit ESPN for <strong>NBA</strong> live scores, video highlights and <strong>latest</strong> <strong>news</strong>. Stream games on ESPN and play Fantasy Basketball.
   https://www.espn.com/nba/

•  NBA (nba.com)
   The official site of the NBA for the latest NBA Scores, Stats & News. | NBA.com
   Follow the action on <strong>NBA</strong> scores, schedules, stats, <strong>news</strong>, teams, and players. Buy tickets or watch the games anywhere with <strong>NBA</strong> League Pass.
   https://www.nba.com/

•  Cbssports (cbssports.

In [ ]:
# agent loop
final_answer = ""

question = "What is the latest news in the NBA? Give me a 3 paragraph report."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

turns = 0
while True:
    turns += 1
    retries = 0 
    while retries < 5:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages, # type: ignore
            )
            response = response.choices[0].message.content # type: ignore
            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore
            if maybe_thinking or maybe_tool_call or maybe_answer:
                break
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
    print("=== Turn", turns, "===")
    if maybe_thinking: # type: ignore
        thinking = maybe_thinking.strip()
        print(f"Thinking: {thinking}")
    if maybe_tool_call: # type: ignore
        tool_call = maybe_tool_call
        tool_result = call_tool(tool_call)
        print(f"Tool call: {tool_call}")
        print(f"Tool result: {tool_result[:100]}")
        messages.append({"role": "user", "content": tool_result})
    elif maybe_answer: # type: ignore
        final_answer = maybe_answer
        break
    else:
        print("Error: No tool call or answer found")
        break

=== Turn 1 ===
Thinking: I will search for the latest NBA news to provide a 3-paragraph summary that covers top stories, player updates, and league developments.
Tool call: {'name': 'search', 'args': {'query': 'latest NBA news'}}
Tool result: •  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong>
=== Turn 2 ===
Thinking: To provide the latest NBA news, I will check the official NBA.com news page for the most updated and reliable information on games, player performances, and league events.
Tool call: {'name': 'fetch', 'args': {'url': 'https://www.nba.com/news'}}
Tool result: NBA News - Latest team, player and league news | NBA.com

Navigation Toggle[![NBA Logo](https://cdn.
=== Turn 3 ===
Thinking: I now have the latest top headlines and news from NBA.com, including playoff results, standout player performances, and major league updates. I can now synthesize this information into a concise three-paragraph report.


In [44]:
print(f"Final answer: {final_answer}")

Final answer: 
The Oklahoma City Thunder have clinched a spot in the NBA Finals for the first time in 13 years, eliminating the Minnesota Timberwolves with an impressive performance characterized by star power, depth, and defense. Shai Gilgeous-Alexander, recently named the 2024-25 Kia NBA MVP, and his young supporting cast, including Jalen Williams and Chet Holmgren, have proven pivotal throughout the playoffs, propelling OKC back to the league’s biggest stage. Their balanced team effort and defensive prowess set a blueprint for postseason success that young teams around the league will look to emulate.

Meanwhile, the Eastern Conference Finals are heating up with the Indiana Pacers taking a commanding 3-1 lead over the New York Knicks. Tyrese Haliburton made history in Game 4, posting a remarkable triple-double with at least 30 points, 15 assists, and 10 rebounds—without a single turnover—a feat never before achieved in NBA playoff history. As Indiana’s offense continues to surge, th

In [46]:
### Models as tools

def ask_model(question: str, url: str) -> str:
    """Ask a model a question about a URL and return the answer."""

    fetch_result = fetch(url)
    system_prompt = "You are a helpful assistant that can answer questions about a given URL."
    prompt = f"""
    Here is the content of the URL {url}:
    {fetch_result}

    Here is the question:
    {question}
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    # ask model to answer question about url
    answer = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages, # type: ignore
    )
    return answer.choices[0].message.content # type: ignore

# test
question = "What is the latest news in the NBA?."
url = "https://www.nba.com/news"
answer = ask_model(question, url)
print(answer)

The latest NBA news includes the following headlines:

1. "5 takeaways: Thunder finish Wolves in 5" - Oklahoma City Thunder advance to the NBA Finals with a strong close-out win over the Minnesota Timberwolves. (5 hours ago)

2. "4 stats to know for Knicks-Pacers Game 5" - Indiana Pacers hold a 3-1 series lead over the New York Knicks, with their offense being a key factor. (Recent)

3. "5 key stats from OKC's run to NBA Finals" - Oklahoma City Thunder have impressive stats as they return to the NBA Finals. (Recent)

4. "Thunder showing blueprint for success" - Isiah Thomas praises the Thunder's organization and suggests Minnesota Timberwolves could learn from them. (Video, 6 minutes duration)

5. "Adelman reveals Nuggets' offseason plans" - New Denver Nuggets coach discusses team conditioning and strategies. (19 hours ago)

6. "5 takeaways: Haliburton dazzles in Game 4" - Tyrese Haliburton posts a triple-double, helping the Indiana Pacers in Game 4 against the Knicks. (15 hours ago)



In [49]:
# 

import os
from openai import OpenAI

model_name = "gpt-4.1"
base_url = "https://api.openai.com/v1"
client = OpenAI(base_url=base_url, api_key=os.getenv("OPENAI_API_KEY"))

system_prompt = """
You are a helpful assistant that can answer questions and help with tasks, such as drafting short research reports. Cite your sources when relevant. 

You have access to the following tools:

- search(query: str) -> str: Searches the web and returns summaries of top results.
- fetch(url: str) -> str: Fetches the content of a given URL and returns it as a markdown page.
- ask_model(question: str, url: str) -> str: Ask an AI helper model a question about a given URL and return the answer.

You may call one tool per turn, for up to 10 turns, before giving your final answer.

If you expect that a web page's content may be fairly long, you should opt to use the `ask_model` tool instead of `fetch`.

In each turn, you should respond in the following format:

<think>
[your thoughts here]
</think>
<tool>
JSON with the following fields:
- name: The name of the tool to call
- args: A dictionary of arguments to pass to the tool (must be valid JSON)
</tool>

When you are done, give your final answer in the following format:

<answer>
[your final answer here]
</answer>
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the latest news in the NBA?"},
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages, # type: ignore
)

response = response.choices[0].message.content # type: ignore
print(response)


<think>
I'll search for the latest news in the NBA to provide the most recent updates, including trades, injuries, game results, or other major headlines.
</think>
<tool>
{
  "name": "search",
  "args": {
    "query": "latest NBA news"
  }
}
</tool>


In [51]:
# updated call_tool
def call_tool(tool_call: dict) -> str:
    """Call a tool with the given tool call."""
    if tool_call['name'] == 'search':
        return search(tool_call['args']['query'])
    elif tool_call['name'] == 'fetch':
        return fetch(tool_call['args']['url'])
    elif tool_call['name'] == 'ask_model':
        return ask_model(tool_call['args']['question'], tool_call['args']['url'])
    else:
        return f"Error: Tool {tool_call['name']} not found"
    
# agent loop
final_answer = ""

question = "What is the latest news in the NBA? Give me a 3 paragraph report."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

turns = 0
while True:
    turns += 1
    retries = 0 
    while retries < 5:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages, # type: ignore
            )
            response = response.choices[0].message.content # type: ignore
            # parse for thinking, tool call, and/or answer 
            maybe_thinking = parse_thinking_from_response(response) # type: ignore
            maybe_tool_call = parse_tool_from_response(response) # type: ignore
            maybe_answer = parse_answer_from_response(response) # type: ignore
            if maybe_thinking or maybe_tool_call or maybe_answer:
                break
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
    print("=== Turn", turns, "===")
    if maybe_thinking: # type: ignore
        thinking = maybe_thinking.strip()
        print(f"Thinking: {thinking}")
    if maybe_tool_call: # type: ignore
        tool_call = maybe_tool_call
        tool_result = call_tool(tool_call)
        print(f"Tool call: {tool_call}")
        print(f"Tool result: {tool_result[:100]}")
        messages.append({"role": "user", "content": tool_result})
    elif maybe_answer: # type: ignore
        final_answer = maybe_answer
        break
    else:
        print("Error: No tool call or answer found")
        break

=== Turn 1 ===
Thinking: I'll search for the latest NBA news to gather current updates and trends. Then, I'll summarize the findings into a concise three-paragraph report.
Tool call: {'name': 'search', 'args': {'query': 'latest NBA news'}}
Tool result: •  NBA (nba.com)
   NBA News - Latest team, player and league news | NBA.com
   <strong>NBA</strong>
=== Turn 2 ===
Thinking: The best way to get the latest, up-to-date NBA news in detail is to check the "Top Stories" or main news page on NBA.com. I will use the "Top Stories" page from NBA.com to ensure official and current news for my report.
Tool call: {'name': 'ask_model', 'args': {'question': 'What are the most important and latest NBA news stories or developments today?', 'url': 'https://www.nba.com/news/category/top-stories'}}
Tool result: The most important and latest NBA news stories and developments today from the NBA.com Top Stories s
=== Turn 3 ===


In [52]:
print(f"Final answer: {final_answer}")

Final answer: 
The latest NBA news centers around the Oklahoma City Thunder's remarkable return to the NBA Finals for the first time in 13 years. Key stats from their playoff run have underscored their defensive tenacity, the emergence of their young stars, and the remarkable versatility of Jalen Williams, who is averaging 20.5 points, 5.6 rebounds, 5.3 assists, and 1.7 steals over 15 postseason games. The Thunder clinched their Finals berth by defeating the Minnesota Timberwolves in five games, leveraging their depth and ability to perform in crucial moments.

As the Western Conference Finals conclude, the focus in the Eastern Conference Finals shifts to the series between the Indiana Pacers and New York Knicks, with the Pacers leading 3-1. Tyrese Haliburton has delivered historic performances, including a triple-double with at least 30 points, 15 assists, 10 rebounds, and zero turnovers—a feat never before accomplished in NBA playoff history. The Knicks face elimination in a pivotal 

### MCP Crash Course

TLDR: MCP is basically just function calling

Why:
- multiple function methods
- client/server architecture, clients + servers can manage their own state
- standardized interface, portability
- both Python + Typescript servers supported (uvx, npx)
    - also Java/Kotlin/C#/Swift SDKs, but less popular

Getting Started:
- MCP Server [docs](https://modelcontextprotocol.io/quickstart/server)
- MCP Client [docs](https://modelcontextprotocol.io/quickstart/client)
- example servers: [modelcontextprotocol/servers](https://github.com/modelcontextprotocol/servers)
- Other repositories:
    - [Smithery](https://smithery.ai/)
    - [mcp.so](https://mcp.so/)
    - [awesome-mcp-servers](https://github.com/punkpeye/awesome-mcp-servers)


MCP Clients:
- Claude Desktop
- Claude Code
- Cursor, Windsurf, other IDEs

### Deep Research via Claude Code

Claude Code is set up to be a code agent by default, but can also be used as a sandbox for other agentic workflows

We just need:
- tools
- instructions
- examples?
- ability to refine via feedback

In [ ]:
basic_prompt = """
Write a report about upcoming tech events in NYC. Save the report as a markdown file in the reports folder.
"""

##